### import all the necessary libraries:

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from xgboost import XGBClassifier
from sklearn.neighbors import KNeighborsClassifier

from sklearn.metrics import accuracy_score
from sklearn.model_selection import GridSearchCV

### Pass the warnings:

In [ ]:
import warnings
warnings.filterwarnings('ignore')


### Read the dataset and store it into pandas dataframe:

In [ ]:
df = pd.read_csv('dft-road-casualty-statistics-accident-2021.csv', low_memory=False)

# Exploratory Data Analysis:

### Displayt the shape of the dataset:

In [ ]:
df.shape

### Display the data types of all columns in the data frame:

In [ ]:
df.dtypes

### Display the column names in the data frame:

In [ ]:
df.columns

### display the general information about the dataset:

In [ ]:
df.info()

### Print the first 10 rows of the dataset:

In [ ]:
df.head(10)

### Convert date into pandas datetime object:

In [ ]:
df['date'] = pd.to_datetime(df['date'],)

### Extract day, month and week some useful features from 'date' column:

In [ ]:
df['day'] = df['date'].dt.day
df['month'] = df['date'].dt.month
df['week'] = df['date'].dt.isocalendar().week

### accident_severity is out target variable, display the unique values in accident_severity column:

### 1 = Satal, 2= Serious and 3 = Slight

In [ ]:
df['accident_severity'].unique()

### display the value count for the accident_severity (target) variable:

In [ ]:
df['accident_severity'].value_counts()

# Univariate Analysis:

### Distribution of original data by target (accident_severity):

In [ ]:
ax = sns.countplot(x = df.accident_severity ,palette="Set2")
sns.set(font_scale=1)
ax.set_xlabel(' ')
ax.set_ylabel(' ')
fig = plt.gcf()
fig.set_size_inches(14,8)
for p in ax.patches:
    ax.annotate('{:.2f}%'.format(100*p.get_height()/len(df.accident_severity)), (p.get_x()+ 0.3, p.get_height()+10000))

plt.title('Distribution of  Targets variable',fontsize = 15)
plt.xlabel('Accident Severity',fontsize = 15)
plt.ylabel('Frequency [%]',fontsize = 15)
plt.show()

### Number of casualties distribution

In [ ]:
plt.figure(figsize=(16,8))
sns.distplot(df.number_of_casualties).set_xlim(0,20)
plt.xlabel('number_of_casualties' ,fontsize = 15)
plt.ylabel('Density',fontsize = 15)
plt.show()
print('Min:',    df.number_of_casualties.min(), '\n'
      'Max:',    df.number_of_casualties.max(), '\n'
      'Median:', df.number_of_casualties.median())

### Distribution of accidents over the day

In [ ]:
time_x = pd.to_datetime(df['time'], format='%H:%M').dt.hour
plt.figure(figsize=(16,8))
ax = time_x.value_counts().sort_index().plot(kind = 'area')
ax.set_xlabel('Hours of the day', fontsize = 15)
ax.set_ylabel('Count of Accidents', fontsize = 15)
ax.set_title('Distribution of accidents over the day', fontsize = 15)
plt.show()

### Number of vehicles distribution

In [ ]:
plt.figure(figsize=(16,8))
sns.distplot(df.number_of_vehicles).set_xlim(0,20)
plt.xlabel('number_of_vehicles',fontsize = 15)
plt.ylabel('Density',fontsize = 15)
plt.show()
print('Min:',    df.number_of_vehicles.min(), '\n'
      'Max:',    df.number_of_vehicles.max(), '\n'
      'Median:', df.number_of_vehicles.median())

### A boxplot to show Distribution of Speed Limit:

In [ ]:
import seaborn as sns
plt.figure(figsize = (16, 8))
sns.boxplot(x=df["speed_limit"])
plt.title("Distribution of Speed Limit",fontsize = 15)
plt.xlabel("Speed Limit",fontsize = 15)
plt.show()


### Bar chart of "day_of_week"

In [ ]:
plt.figure(figsize = (16, 8))
sns.countplot(x=df["day_of_week"])
plt.title("Number of Accidents by Day of Week",fontsize = 15)
plt.xlabel("Day of Week",fontsize = 15)
plt.ylabel("Count",fontsize = 15)
plt.show()

# Multivariate Analysis:

### Scatter plot of "speed_limit" vs "number_of_casualties"

In [ ]:
plt.figure(figsize = (16, 8))
plt.scatter(x=df["speed_limit"], y=df["number_of_casualties"])
plt.title("Speed Limit vs Number of Casualties",fontsize = 15)
plt.xlabel("Speed Limit",fontsize = 15)
plt.ylabel("Number of Casualties",fontsize = 15)
plt.show()


### Box plot of "speed_limit" by "weather_conditions"

In [ ]:
plt.figure(figsize = (16, 8))
sns.boxplot(x=df["weather_conditions"], y=df["speed_limit"])
plt.title("Speed Limit by Weather Conditions",fontsize = 15)
plt.xlabel("Weather Conditions",fontsize = 15)
plt.ylabel("Speed Limit",fontsize = 15)
plt.show()


### Drop the unnecessary columns:

In [ ]:
df.drop(['accident_index', 'accident_reference', 'local_authority_ons_district', 'local_authority_highway','time', 'date', 'lsoa_of_accident_location','accident_year'], axis = 1, inplace = True)

### Check are there any missing values in the data frame:

In [ ]:
df.isnull().any()

### Count the number of missing values in each column of the data frame:

In [ ]:
df.isnull().sum()

### Fill the missing values in location_easting_osgr, location_northing_osgr, longitude and latitude columns:

In [ ]:
df['location_easting_osgr'] = df['location_easting_osgr'].fillna(df['location_easting_osgr'].mean())
df['location_northing_osgr'] = df['location_northing_osgr'].fillna(df['location_northing_osgr'].mean())
df['longitude'] = df['longitude'].fillna(df['longitude'].mode()[0])
df['latitude'] = df['latitude'].fillna(df['latitude'].mode()[0])

### Convert (1 = Satal, 2= Serious and 3 = Slight) to binary values (0 = Slight, 1 =  Serious): 
### Now we have binary classification problem

In [ ]:
df['accident_severity'] = df['accident_severity'].replace([2 , 1], 0)
df['accident_severity'] = df['accident_severity'].replace(3, 1)

### Value counts of the accident_severity again:

In [ ]:
df['accident_severity'].value_counts()

### Distribution of original data by targets


In [ ]:

ax = sns.countplot(x = df.accident_severity ,palette="Set2")
sns.set(font_scale=1)
ax.set_xlabel(' ')
ax.set_ylabel(' ')
fig = plt.gcf()
fig.set_size_inches(14,8)
for p in ax.patches:
    ax.annotate('{:.2f}%'.format(100*p.get_height()/len(df.accident_severity)), (p.get_x()+ 0.3, p.get_height()+10000))

plt.title('Distribution of  Targets variable',fontsize = 15)
plt.xlabel('Accident Severity',fontsize = 15)
plt.ylabel('Frequency [%]',fontsize = 15)
plt.show()

### Store features into feature matrix X and target into vector y:

In [ ]:
# Features
X = df.loc[:,df.columns != 'accident_severity']
#Target
y = df['accident_severity']

## Feature Selection using SelectKbest method:

### import SelectKBest and f_classif from sklear for feature seletion:

In [ ]:
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import f_classif

### Initialized SelectKBest with f_classif test and k= 15

In [ ]:
test = SelectKBest(score_func=f_classif, k=15)

### Fit the SelectKBest  model:

In [ ]:
fit = test.fit(X, y)

### Transform the features:

In [ ]:
filtered_features= fit.transform(X)

### Show the supporting variables given by SelectKBest (True/false):

In [ ]:
test.get_support()

### Display the scores of the features assigned by the SelectkBest:

In [ ]:
test.scores_

### Find the Feature importance for each feature:

In [ ]:
feat_importances = pd.Series(test.scores_, index=X.columns)

### Display the feature importance using a bar chart

In [ ]:
feat_importances.nlargest(15).plot(kind='barh',figsize = (16,8))
plt.show()

### Getting the column names:

In [ ]:
column_names = X.columns[test.get_support()]
column_names

### Selecting the  relevant columns given by SelectKbest only: 

In [ ]:
X_Selectkbest = X[column_names]
X_Selectkbest.head()

### Divide the data set into training and Testing sets (training set 80%, Testing set 20%):

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X_Selectkbest, y, test_size=0.20, random_state=0)  

### Use StandardScaler to scale the values of the dataset:

In [ ]:
SC = StandardScaler()
X_train_scaled = SC.fit_transform(X_train)
X_test_scaled = SC.fit_transform(X_test)

# 	Experimental Design:

### Logistic Regression Model

In [ ]:
#Initialize the LogisticRegression Model
Lg_classifier = LogisticRegression()
# Train the model on training set
Lg_classifier.fit(X_train_scaled, y_train)

### Make prediction and evaluation on seen data:

In [ ]:
# Make prediction on seen data
y_pred = Lg_classifier.predict(X_train_scaled)
#he Accuracy score for Logistic Regresson on seen data :
Lg_score_seen = accuracy_score(y_train, y_pred)
print('The Accuracy score for Logistic Regresson on seen data : ', Lg_score_seen)

### Make prediction and evaluation on unseen data:

In [ ]:
# Make prediction on unseen data
y_pred = Lg_classifier.predict(X_test_scaled)
#he Accuracy score for Logistic Regresson on unseen data :
Lg_score_unseen = accuracy_score(y_test, y_pred)
print('The Accuracy score for Logistic Regresson on unseen data : ', Lg_score_unseen)


### Decision Tree Classifier Model

In [ ]:
#Initialize the DecisionTreeClassifier Model
Dt_classifier = DecisionTreeClassifier()
# Train the model on training set
Dt_classifier.fit(X_train_scaled, y_train)

### Make prediction and evaluation on seen data:

In [ ]:
# Make prediction on seen data
y_pred = Dt_classifier.predict(X_train_scaled)
#he Accuracy score for decision tree on seen data :
Dt_score_seen = accuracy_score(y_train, y_pred)
print('The Accuracy score for decision tree on seen data : ', Dt_score_seen)

### Make prediction and evaluation on unseen data:

In [ ]:
# Make prediction on unseen data
y_pred = Dt_classifier.predict(X_test_scaled)
#he Accuracy score for decision tree on unseen data :
Dt_score_unseen = accuracy_score(y_test, y_pred)
print('The Accuracy score for decision tree on unseen data : ', Dt_score_unseen)


### RandomForest Classifier Model

In [ ]:
#Initialize the RandomForestClassifier Model
Rf_Classifier = RandomForestClassifier(n_estimators = 100, )
# Train the model on training set
Rf_Classifier.fit(X_train_scaled, y_train)

### Make prediction and evaluation on seen data:

In [ ]:
# Make prediction on seen data
y_pred = Rf_Classifier.predict(X_train_scaled)
#he Accuracy score for Random Forest on seen data :
Rf_score_seen = accuracy_score(y_train, y_pred)
print('The Accuracy score for Random Forest on seen data : ', Rf_score_seen)

### Make prediction and evaluation on unseen data:

In [ ]:
# Make prediction on unseen data
y_pred = Rf_Classifier.predict(X_test_scaled)
#he Accuracy score for Random Forest on unseen data :
Rf_score_unseen = accuracy_score(y_test, y_pred)
print('The Accuracy score for Random Forest on unseen data : ', Rf_score_unseen)


### Gradient Boosting Classifier Model

In [ ]:
#Initialize the GradientBoostingClassifier Model
Gb_Classifier = GradientBoostingClassifier()
# Train the model on training set
Gb_Classifier.fit(X_train_scaled, y_train)

### Make prediction and evaluation on seen data:

In [ ]:
# Make prediction on seen data
y_pred = Gb_Classifier.predict(X_train_scaled)
#he Accuracy score for Gradient Boosting on seen data :
Gb_score_seen = accuracy_score(y_train, y_pred)
print('The Accuracy score for Gradient Boosting on seen data : ', Gb_score_seen)

### Make prediction and evaluation on unseen data:

In [ ]:
# Make prediction on unseen data
y_pred = Gb_Classifier.predict(X_test_scaled)
#he Accuracy score for  Gradient Boosting on unseen data :
Gb_score_unseen = accuracy_score(y_test, y_pred)
print('The Accuracy score for  Gradient Boosting on unseen data : ', Gb_score_unseen)


### XGB Classifier Model

In [ ]:
#Initialize the XGBClassifier Model
XGB_Classifier = XGBClassifier()
# Train the model on training set
XGB_Classifier.fit(X_train_scaled, y_train)

### Make prediction and evaluation on seen data:

In [ ]:
# Make prediction on seen data
y_pred = XGB_Classifier.predict(X_train_scaled)
#he Accuracy score for XGB_Classifier on seen data :
XGB_score_seen = accuracy_score(y_train, y_pred)
print('The Accuracy score for XGB_Classifier on seen data : ', XGB_score_seen)

### Make prediction and evaluation on unseen data:

In [ ]:
# Make prediction on unseen data
y_pred = XGB_Classifier.predict(X_test_scaled)
#he Accuracy score for  XGB_Classifier on unseen data :
XGB_score_unseen = accuracy_score(y_test, y_pred)
print('The Accuracy score for XGB_Classifier on unseen data : ', XGB_score_unseen)


### KNeighbors Classifier Model

In [ ]:
#Initialize the KNeighborsClassifier Model
KNN_Classifier = KNeighborsClassifier()
# Train the model on KNeighborsClassifier set
KNN_Classifier.fit(X_train_scaled, y_train)

### Make prediction and evaluation on seen data:

In [ ]:
# Make prediction on seen data
y_pred = KNN_Classifier.predict(X_train_scaled)
#he Accuracy score for KNN classifier on seen data :
KNN_score_seen = accuracy_score(y_train, y_pred)
print('The Accuracy score for KNN_Classifier on seen data : ', KNN_score_seen)

### Make prediction and evaluation on unseen data:

In [ ]:
# Make prediction on unseen data
y_pred = KNN_Classifier.predict(X_test_scaled)
#he Accuracy score for  KNN classifier on unseen data :
KNN_score_unseen = accuracy_score(y_test, y_pred)
print('The Accuracy score for KNN_Classifier on unseen data : ', KNN_score_unseen)


### Create two lists of model name and respective Accuracies:

In [ ]:
scores = pd.Series([Lg_score_unseen, Dt_score_unseen, Rf_score_unseen, Gb_score_unseen, XGB_score_unseen, KNN_score_unseen,  ])
Model_Names = ['Logistic Regression','DecisionTree Classifier','RandomForest Classifier', 'GBR Classifier' ,'XGB Classifier' ,'KNN Classifier',]

### Comparision of Models  in terms  evaluation metric (accuracy Score): 

In [ ]:
ax = scores.plot(kind = 'barh',figsize=(15,9),color=['black','gray','green','brown','pink','blue','red'])
ax.set_title('Comparision of Models (Accuracy) unseen data',fontsize=15)
ax.set_xticks([0.0,0.1,0.2,0.3,0.4,0.5,0.6,0.7,0.8,0.9,1.0])
ax.set_yticklabels(Model_Names,fontsize=15,)
ax.set_ylabel("Models",fontsize=15)
ax.set_xlabel("Accuracy",fontsize=15)
[ax.text(v, i, '{:.2f}%'.format(100*v)) for i, v in enumerate(scores)];
plt.show()

# 	Evaluation and further modelling improvements

### Hyper parameter Tuning for Highest performing Model (Logistic Regression)

In [ ]:
# Create logistic regression object
lr = LogisticRegression()

# Set hyperparameters to tune
hyperparameters = {
    'penalty': ['l1', 'l2'],
    'C': [0.01, 0.1, 1, 10, 100],
    'solver': ['liblinear']
}

###  Create GridSearchCV object and fit it on training data:

In [ ]:
# Create GridSearchCV object
clf = GridSearchCV(lr, hyperparameters, cv=5)

# Fit GridSearchCV object to training data
clf.fit(X_train_scaled, y_train)


### Print best hyperparameters and best score

In [ ]:
# Print best hyperparameters and best score
print('Best parameters:', clf.best_params_)
print('Best score:', clf.best_score_)
